# A Curious Case

Arvindh gave me the following code yesterday. At first glance, what do you think it does?

You may need to build with make LDFLAGS="-L/opt/homebrew/opt/gettext/lib -lintl"

In [4]:

x = (1, 2, [30, 40])
try:
    x[2] += [50, 60]
except Exception as e:
    print("Exception! " + str(e))
    pass
print(x)

Exception! 'tuple' object does not support item assignment
(1, 2, [30, 40, 50, 60])


It may be hard to guess that the code both performs the array addition at `x[2]` AND throws an error. 

Why is that?

Let's look at the following bytecode:

In [8]:
from bytecode import Bytecode, Instr

bytecode = Bytecode([
    Instr("LOAD_CONST", 1),
    Instr("LOAD_CONST", 2),
    Instr("LOAD_CONST", 30),
    Instr("LOAD_CONST", 40),
    Instr("BUILD_LIST", 2),
    Instr("BUILD_TUPLE", 3),
    Instr("STORE_NAME", "t"),
    Instr("LOAD_NAME", "t"),
    Instr("LOAD_CONST", 2),
    Instr("COPY", 2),
    Instr("COPY", 2),
    Instr("BINARY_SUBSCR"),
    Instr("LOAD_CONST", 50),
    Instr("LOAD_CONST", 60),
    Instr("BUILD_LIST", 2),
    Instr("BINARY_OP", 13),  # 13 represents '+='
    Instr("SWAP", 3),
    Instr("SWAP", 2),
    Instr("STORE_SUBSCR"),
    Instr("RETURN_CONST", None)
])

try:
    print(eval(bytecode.to_code()))
except Exception as e:
    print("Error! " + str(e))
print(x)

Error! 'tuple' object does not support item assignment
(1, 2, [30, 40, 50, 60])


It looks like the array is concatenated with the `BUILD_LIST` opcode, then swapped back into the tuple with `STORE_SUBSCR`. `STORE_SUBSCR` is the instruction that throws the error, as a tuple's elements cannot be re-assigned.

What if we remove STORE_SUBSCR?

In [9]:
from bytecode import Bytecode, Instr

bytecode = Bytecode([
    Instr("LOAD_CONST", 1),
    Instr("LOAD_CONST", 2),
    Instr("LOAD_CONST", 30),
    Instr("LOAD_CONST", 40),
    Instr("BUILD_LIST", 2),
    Instr("BUILD_TUPLE", 3),
    Instr("STORE_NAME", "t"),
    Instr("LOAD_NAME", "t"),
    Instr("LOAD_CONST", 2),
    Instr("COPY", 2),
    Instr("COPY", 2),
    Instr("BINARY_SUBSCR"),
    Instr("LOAD_CONST", 50),
    Instr("LOAD_CONST", 60),
    Instr("BUILD_LIST", 2),
    Instr("BINARY_OP", 13),  # 13 represents '+='
    Instr("SWAP", 3),
    Instr("SWAP", 2),
    Instr("RETURN_CONST", None)
])

try:
    print(eval(bytecode.to_code()))
    print("No error!")
except Exception as e:
    print("Error! " + str(e))
print(x)

None
No error!
(1, 2, [30, 40, 50, 60])


Now, the code works - error free!